In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
import json
import dask.dataframe as dd

Regular pandas wasn't working. Learned about dask and it worked beautifully.

In [8]:
seattle_df = dd.read_csv('..\data\checkouts_by_title.csv', dtype={'ISBN': 'object'})

In [9]:
filtered_seattle_df = seattle_df[seattle_df['CheckoutYear'] >= 2018]

In [10]:
filtered_seattle_df.head()

F:\Anaconda\lib\site-packages\dask\dataframe\core.py:7560: UserWarning: Insufficient elements for `head`. 5 elements requested, only 2 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


,UsageClass,CheckoutType,MaterialType,CheckoutYear,CheckoutMonth,Checkouts,Title,ISBN,Creator,Subjects,Publisher,PublicationYear
184756,Digital,OverDrive,EBOOK,2023,3,2,Harry Potter und der Stein der Weisen,9781781100769,J. K. Rowling,"Juvenile Fiction, Juvenile Literature",Pottermore,2020
244526,Digital,OverDrive,AUDIOBOOK,2022,9,2,"The Free Bastards: The Lot Lands Series, Book ...",9780593414682,Jonathan French,"Fantasy, Fiction",Books on Tape,2021


In [11]:
result = filtered_seattle_df.compute()

In [12]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12415765 entries, 184756 to 244722
Data columns (total 12 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   UsageClass       object
 1   CheckoutType     object
 2   MaterialType     object
 3   CheckoutYear     int64 
 4   CheckoutMonth    int64 
 5   Checkouts        int64 
 6   Title            object
 7   ISBN             object
 8   Creator          object
 9   Subjects         object
 10  Publisher        object
 11  PublicationYear  object
dtypes: int64(3), object(9)
memory usage: 1.2+ GB


Saving to a file with just my filtered data.

In [14]:
result.to_csv('..\\data\\filtered_checkouts_by_title.csv', index=False)

In [15]:
result.head()

,UsageClass,CheckoutType,MaterialType,CheckoutYear,CheckoutMonth,Checkouts,Title,ISBN,Creator,Subjects,Publisher,PublicationYear
184756,Digital,OverDrive,EBOOK,2023,3,2,Harry Potter und der Stein der Weisen,9781781100769,J. K. Rowling,"Juvenile Fiction, Juvenile Literature",Pottermore,2020
244526,Digital,OverDrive,AUDIOBOOK,2022,9,2,"The Free Bastards: The Lot Lands Series, Book ...",9780593414682,Jonathan French,"Fantasy, Fiction",Books on Tape,2021
77600,Digital,OverDrive,EBOOK,2022,9,8,"Dawn: Warriors: The New Prophecy Series, Book 3",9780061757433,Erin Hunter,"Fantasy, Juvenile Fiction, Juvenile Literature",HarperCollins Publishers Inc.,2007
131103,Physical,Horizon,BOOK,2022,9,1,Black box : a record of the catastrophe. Volum...,"162963123X, 9781629631233",NaN,"Social change, Social change Literary collections","PM Press,",[2015]
184573,Digital,OverDrive,AUDIOBOOK,2023,3,2,Blood Games (unabridged),9781452698052,Chloe Neill,"Fantasy, Fiction, Literature, Romance","Tantor Media, Inc",2015


Trying to clean ISBN and split into ISBN-10 and ISBN-13.

In [ ]:
result[['Possible ISBN-10', 'Possible ISBN-13']] = result['ISBN'].str.split(',', expand=True)

Fixing any spaces left.

In [ ]:
result['Possible ISBN-10'] = result['Possible ISBN-10'].str.strip()
result['Possible ISBN-13'] = result['Possible ISBN-13'].str.strip()